In [ ]:
import numpy as np
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.problems import QuadraticProgram
from qiskit.providers.ibmq import least_busy
from qiskit import Aer, transpile, IBMQ


##########################
import matplotlib.pyplot as plt
import networkx as nx
from qiskit.providers.ibmq import least_busy
from qiskit import Aer, transpile, IBMQ
from qiskit.circuit.library import TwoLocal
from qiskit_optimization.applications import Tsp
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import VQE
from qiskit.utils import algorithm_globals
from qiskit.algorithms.optimizers import SPSA
from qiskit_ibm_provider import IBMProvider 
#######################################################
from qiskit_ibm_provider import IBMProvider

# Save your credentials on disk.
IBMQ.save_account("4d1458bb120275b4396ee321a6e7cdd76b0a451ed985590b1fbf4e53a60abf33f626bf3944cc35c174f3d7dd79162575f44d8f43e6b320dc4b6b452d4b20cd3c",overwrite=True )
IBMQ.load_account()
provider = IBMProvider(instance='ibm-q-asu/main/tec-german')
backend = provider.get_backend('ibm_brisbane')



# Define the coordinates of the cities
coordinates = np.array([[196.268975,1978.65365], [1111.551625,903.481175], [1726.041125,1175.81045], [1721.44055,615.3756595]])

# Calculate the distances between the cities (using Euclidean distance)
def calculate_distance(coord1, coord2):
    return ((coord1[0] - coord2[0]) ** 2 + (coord1[1] - coord2[1]) ** 2) ** 0.5

num_cities = len(coordinates)
distance_matrix = np.zeros((num_cities, num_cities))

for i in range(num_cities):
    for j in range(num_cities):
        distance_matrix[i][j] = calculate_distance(coordinates[i], coordinates[j])

# Create a quadratic optimization problem for the TSP
problem = QuadraticProgram()
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            problem.binary_var(f'x_{i}_{j}')

# Constraints: each city must be visited exactly once
for i in range(num_cities):
    constraint = {f'x_{i}_{j}': 1 for j in range(num_cities) if j != i}
    problem.linear_constraint(linear=constraint, sense='E', rhs=1, name=f'visit_once_{i}')

# Constraints: from each city, there should be exactly one transition
for j in range(num_cities):
    constraint = {f'x_{i}_{j}': 1 for i in range(num_cities) if i != j}
    problem.linear_constraint(linear=constraint, sense='E', rhs=1, name=f'leave_once_{j}')

# Objective function: minimize the total distance
linear_coeff = {f'x_{i}_{j}': distance_matrix[i][j] for i in range(num_cities) for j in range(num_cities) if i != j}
quadratic_coeff = {}
problem.minimize(linear=linear_coeff, quadratic=quadratic_coeff)

# Solve the problem using the QAOA algorithm on the IBM Quantum backend
qaoa = QAOA(quantum_instance=QuantumInstance(backend, shots=8192))  # Adjust shots as needed
optimizer = MinimumEigenOptimizer(qaoa)

# Solve the problem
result = optimizer.solve(problem)
print('Solution found:', result)

# Map binary variable names to indices
var_to_indices = {(i, j): idx for idx, (i, j) in enumerate([(i, j) for i in range(num_cities) for j in range(num_cities) if i != j])}

# Extract the selected route from the binary variables
selected_route = []
for (i, j), idx in var_to_indices.items():
    if result.x[idx] == 1:
        selected_route.append((i, j))

# Print the selected route
print('Selected Route:', selected_route)

# Visualize the histogram of the solutions
plot_histogram(dict(enumerate(result.x)), figsize=(8, 6), bar_labels=False)
